# Solver Review


In [1]:
import lamberthub

print(f"Current lamberthub version is {lamberthub.__version__}")

Current lamberthub version is 0.1


In [2]:
from lamberthub import ALL_SOLVERS

# - list solvers
print([solver.__name__ for solver in ALL_SOLVERS])
solver_set = [solver.__name__ for solver in ALL_SOLVERS]
for i in solver_set:
    print("Solver name:", i)


['gooding1990', 'avanzini2008', 'arora2013', 'izzo2015']
Solver name: gooding1990
Solver name: avanzini2008
Solver name: arora2013
Solver name: izzo2015


In [1]:
# 2024-06-26, never figured a way to import vallado2013
# import lamberthub.universal_solvers.vallado # does NOT work
# from lamberthub.universal_solvers.vallado import vallado2013 # does NOT work
# from src.lamberthub.universal_solvers.vallado import vallado2013 # does NOT work
# import lamberthub # not needed
from lamberthub import izzo2015 # this works

# from lamberthub import ALL_SOLVERS
import numpy as np # NumPy for declaring position vectors

mu_sun = 39.47692641  # [AU ** 3 / year ** 2]
r1 = np.array([0.159321004, 0.579266185, 0.052359607])  # [AU]
r2 = np.array([0.057594337, 0.605750797, 0.068345246])  # [AU]
tof = 0.010794065  # [year]

# v1, v2 = vallado2013(mu_sun, r1, r2, tof) # this does NOT work
v1, v2 = izzo2015(mu_sun, r1, r2, tof) # this works
print(
    f"izzo2015:\n    initial velocity: {v1} [AU / year]\n    final velocity: {v2} [AU / year]"
)

izzo2015:
    initial velocity: [-9.303608    3.01862016  1.53636008] [AU / year]
    final velocity: [-9.5111862   1.88884006  1.4213781 ] [AU / year]


In [2]:
# - Compare solitions between solution methods
from lamberthub import ALL_SOLVERS

# Import a solver of your choice from the ones listed above
# from lamberthub import authorYYYY
# v1, v2 = authorYYYY(mu, r1, r2, tof, prograde=True, low_path=True,\
#     maxiter=35, atol=1e-5, rtol=1e-7, full_output=False)
from lamberthub import gooding1990, avanzini2008, arora2013, izzo2015
import numpy as np  # - NumPy for declaring position vectors

# - Problem initial conditions for each solution method
#   From An Introduction to the Mathematics and Methods of Astrodynamics, revised edition
#   by R.H. Battin, problem 7-12.
mu_sun = 39.47692641  # [AU ** 3 / year ** 2]
r1 = np.array([0.159321004, 0.579266185, 0.052359607])  # [AU]
r2 = np.array([0.057594337, 0.605750797, 0.068345246])  # [AU]
tof = 0.010794065  # [year]

# - Compare solitions between solution methods.
#   Commented out section manually calls solution method. Loop below calls each method.
# v1, v2 = gooding1990(mu_sun, r1, r2, tof)
# print(f"gooding1990:\n    initial velocity: {v1} [AU / years]\n    final velocity: {v2} [AU / years]")
# v1, v2 = avanzini2008(mu_sun, r1, r2, tof)
# print(f"avanzini2008:\n    initial velocity: {v1} [AU / years]\n    final velocity: {v2} [AU / years]")
# v1, v2 = arora2013(mu_sun, r1, r2, tof)
# print(f"arora2013:\n    initial velocity: {v1} [AU / years]\n    final velocity: {v2} [AU / years]")
# v1, v2 = izzo2015(mu_sun, r1, r2, tof)
# print(f"izzo2015:\n    initial velocity: {v1} [AU / years]\n    final velocity: {v2} [AU / years]")

solver_set = [solver.__name__ for solver in ALL_SOLVERS]
# - below, 2024-02-29, not sure I understand what globals() commmand does; but it gets the job done for now
# - https://stackoverflow.com/questions/3061/calling-a-function-of-a-module-by-using-its-name-a-string
for i in solver_set:
    v1, v2 = globals()[i](mu_sun, r1, r2, tof)
    print(
        f"{i}:\n    initial velocity: {v1} [AU / years]\n    final velocity: {v2} [AU / years]"
    )

gooding1990:
    initial velocity: [-9.303608    3.01862016  1.53636008] [AU / years]
    final velocity: [-9.5111862   1.88884006  1.4213781 ] [AU / years]
avanzini2008:
    initial velocity: [-9.303608    3.01862016  1.53636008] [AU / years]
    final velocity: [-9.5111862   1.88884006  1.4213781 ] [AU / years]
arora2013:
    initial velocity: [-9.30360599  3.01861977  1.53635978] [AU / years]
    final velocity: [-9.51118423  1.88883943  1.42137778] [AU / years]
izzo2015:
    initial velocity: [-9.303608    3.01862016  1.53636008] [AU / years]
    final velocity: [-9.5111862   1.88884006  1.4213781 ] [AU / years]


# Earth to Venus Mission


In [1]:
# - AIAA Brown p.107
from lamberthub import gooding1990, avanzini2008, arora2013, izzo2015
import numpy as np  # - NumPy for declaring position vectors
from astropy import units as u  # for units conversion


# - vector angle for flight path angle; in next section
def vec_angle_deg(a: np.ndarray, b: np.ndarray) -> float:
    n_a = np.linalg.norm(a, axis=0)
    n_b = np.linalg.norm(b, axis=0)
    y = np.linalg.norm(n_b * a - n_a * b, axis=0)
    x = np.linalg.norm(n_b * a + n_a * b, axis=0)
    return np.rad2deg(2 * np.arctan2(y, x))


# - Problem initial conditions taken from my skyfield program; SF-Ephemeris.ipynb
mu_sun = 39.47692641  # [AU ** 3 / year ** 2]
tof = 109 / 365.25  # time of flight [year]
r1 = np.array([-0.95300372, -0.29020189, -0.12581142])  # earth position [AU]
r2 = np.array([0.62867429, -0.31143088, -0.18011123])  # venus position [AU]

v1, v2 = izzo2015(mu_sun, r1, r2, tof)
print(
    f"izzo2015:\n    initial velocity: {v1} [AU / years]\n    final velocity: {v2} [AU / years]"
)

# - convert units in preparation for COE conversion
vUnits_Earth = (v1 * u.AU / u.year).to(u.km / u.s)
vUnits_Venus = (v2 * u.AU / u.year).to(u.km / u.s)
print("converting velocity units, with astropy:")
print("    transfer v at Earth:", np.linalg.norm(vUnits_Earth))
print("    transfer v at Venus:", np.linalg.norm(vUnits_Venus))

izzo2015:
    initial velocity: [ 2.13703812 -4.68371903 -2.50892076] [AU / years]
    final velocity: [3.38904525 6.40764365 3.25998621] [AU / years]
converting velocity units:
    transfer v at Earth: 27.14880277478502 km / s
    transfer v at Venus: 37.67735791093143 km / s


In [5]:
# - flight path angle
flightAngle_launch = vec_angle_deg(r1, v1)
print("launch flight path angle:", 90 - flightAngle_launch)
flightAngle_arrive = vec_angle_deg(r2, v2)
print("arrive flight path angle:", 90 - flightAngle_arrive)

launch flight path angle: -3.60643769572539
arrive flight path angle: -4.504008404355915


# Earth to Mars Mission<font size=2>

May want to review the workflow in the link below,  
2024-03-04, https://letstalkscience.ca/sites/default/files/2021-02/trip-to-mars-solutions.pdf


In [ ]:
# may be worth reviewing the rest of the code from the hyperlink above
import numpy as np
import astropy as ap
import math
from matplotlib import pyplot as plt
import HelperFunctions as h

## *** CELL 2.1  *** ##
# 1) First Run the cell to see Earth's orbit!
# This parameter is intialitizing the plot axis limits (see 'HelperFunctions.py' file to see how it's used)
max_major = 0
# Here we are calling to the python plotting library 'matplotlib' to intialize an empty figure of size 8x8
fig, ax = plt.subplots(figsize=[8, 8])
# Now we will call on helper code from our HelperFunctions, 'h' library.
# 2) Try playing around with the 'e' parameter. REMEMBER: e has to be a number between 0 and 1
# For example: you can try e=0.2, 0.4, 0.6, 0.8, 0.999, 1
h.plot_orbit(major=1, e=0.0167, color="cyan", ax=ax, max_major=max_major)
# What value of e gives a perfect circle? What about a line?
# 3) Try playing around with the 'major' parameter
# 4) Uncomment the line below. For example: you can try major=0, 0.5, 1, 5, 10
h.plot_orbit(major=1, e=0, color="indigo", ax=ax, max_major=max_major)
# What does the parameter 'major' control? (Hint: make sure to look at the axis values when you change major)

# Convert R & V to COE (classic orbital elements) <font size=2>

2024-02-29, https://orbital-mechanics.space/classical-orbital-elements/orbital-elements-and-the-state-vector.html  
2024-03-01, kepler checkout, https://github.com/aerospaceresearch/orbitdeterminator/blob/master/orbitdeterminator/kep_determination/orbital_elements.py  
2024-03-01, space system notes, https://github.com/kyleniemeyer/space-systems-notes  
2024-03-01, reasonable theory, https://orbital-mechanics.space/intro/reference-frames.html


In [1]:
# Step 1—Position and Velocity Magnitudes
import numpy as np

# R & V earth data 1988-04-08 from Horizons (and skyfield)
# r_vec = [-1.409206342323255E+08, 4.884114655615644E+07, 2.926503946314380E+04]
# v_vec = [-1.039747672698875E+01, -2.820517858596729E+01, 2.495229861189330E-03]
r_vec = [-1.42567328e08, -4.34135851e07, -1.88211204e07]  # - [km]
v_vec = [8.98201364, -26.02254781, -11.28304798]  # - [km/s]
# - R & V earth data 1988-04-08 from Curtis, p.212
# r_vec = [-6045, -3490, 2500]  # [km]
# v_vec = [-3.457, 6.618, 2.533]  #[km/s]
# remember, double parentheses bundles things...
# r_vec = np.array((-1.42019455e+08, -4.37089122e+07, -1.89514749e+07))  # [km]
# v_vec = np.array((8.97691931, -26.01359291, -11.27913009))  # [km/s]
# r_vec = np.array((1000, 5000, 7000))  # km
# v_vec = np.array((3.0, 4.0, 5.0))  # km/s
# mu = 3.98600e5  # [km^3/s^2], earth
mu = 1.32712440018e11  # [km^3/s^2], sun

r = np.linalg.norm(r_vec)
v = np.linalg.norm(v_vec)
v_r = np.dot(r_vec / r, v_vec)
v_p = np.sqrt(v**2 - v_r**2)
print("r=", r, "[km], v=", v, "[km/s]")
print("v_r=", v_r, "[km/s], v_p=", v_p, "[km/s]")

r= 150214569.72413105 [km], v= 29.75158374392637 [km/s]
v_r= 0.40974729533555276 [km/s], v_p= 29.748762031819563 [km/s]


In [2]:
# - Step 2—Orbital Angular Momentum
h_vec = np.cross(r_vec, v_vec)
h = np.linalg.norm(h_vec)
print("h_vec=", h_vec, "h=", h)

h_vec= [ 6.40582203e+04 -1.77764556e+09  4.09990652e+09] h= 4468697488.435342


In [3]:
# - Step 3—Inclination
i = np.arccos(h_vec[2] / h)
print("inclination:", i * 180 / np.pi, "[deg]")
print("inclination:", i, "[rad]")

inclination: 23.440685600809765 [deg]
inclination: 0.40911714265895555 [rad]


In [4]:
# - Step 4—Right Ascension of the Ascending Node
K = np.array((0, 0, 1))
N_vec = np.cross(K, h_vec)
N = np.linalg.norm(N_vec)
Omega = np.arccos(N_vec[0] / N)
if N_vec[1] < 0:
    Omega = 2 * np.pi - Omega
print("omega=", Omega * 180 / np.pi, "[deg]")
print("omega=", Omega, "[rad]")
print("Ny=", N_vec[1])

omega= 0.0020646780038813383 [deg]
omega= 3.603542916123361e-05 [rad]
Ny= 64058.22034680843


In [5]:
# - Step 5—Eccentricity
# K = np.array((0, 0, 1))
# N_vec = np.cross(K, h_vec)
# N = np.linalg.norm(N_vec)
# Omega = 2 * np.pi - np.arccos(N_vec[0] / N)

# [section-5]
e_vec = (np.cross(v_vec, h_vec) / mu) - (r_vec / r)
e = np.linalg.norm(e_vec)

print("e_vec=", e_vec, "\ne_mag=", e)

e_vec= [-0.00596073  0.01152226  0.00499594] 
e_mag= 0.013901516080398604


In [6]:
# - Step 6—Argument of Periapsis
omega = 2 * np.pi - np.arccos(np.dot(N_vec, e_vec) / (N * e))

In [7]:
# - Step 7—True Anomaly
nu = np.arccos(np.dot(r_vec / r, e_vec / e))

In [8]:
# - Time Since Periapsis
#   https://orbital-mechanics.space/time-since-periapsis-and-keplers-equation/time-since-periapsis.html

### Misc Plotting<font size=2>


In [ ]:
# Plotting the Earth and Jupiter orbits
# 2024-03-04, seems to work except for the anitmation
# https://stackoverflow.com/questions/72706445/python-animation-assumes-the-wrong-focal-point-of-an-ellipse-how-to-changet-it
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from matplotlib import rcParams

Earth = plt.Circle((0, 0), radius=1.0, fill=False, color="blue")
Jupiter = plt.Circle(
    (0, 0), radius=5.2, fill=False, color="green", linewidth=2
)

# Moving Earth, Jupiter, and Spacecraft
patch_E = plt.Circle((0.0, 0.0), radius=0.05, fill=True, color="blue")
patch_J = plt.Circle((0.0, 0.0), radius=0.14, fill=True, color="green")
patch_H = plt.Circle((0.0, 0.0), radius=0.03, fill=True, color="red")

# equations for the path calcualtion of the ellipse
ra = 5.2  # orbits radius
rp = 1.0  # units [AU] = astronomical units
sun_const = 0.00029  # AU^3/ (days)^2
a = (ra + rp) / 2  # semi major axis
e = (ra - rp) / (ra + rp)  # eccentricity

# 1AU = 149597870691 m
Period = (
    2 * np.pi * np.sqrt((a**3) / (sun_const))
)  # Time it takes for a full ellipse path, the half
# is for the transfer
alpha = 180 - (
    360 * Period / (2 * 4383)
)  # angle for jupiter to be ahead for the transfer, 4383


# days, the days it takes jupiter to orbit the sun, Period should be divided by 2
def init():
    patch_E.center = (0.0, 0.0)
    ax.add_patch(patch_E)
    patch_J.center = (0.0, 0.0)
    ax.add_patch(patch_J)
    patch_H.center = (0.0, 0.0)
    ax.add_patch(patch_H)
    return (
        patch_E,
        patch_J,
        patch_H,
    )


def animate(i):  # Animation function, i*10 makes the animation faster
    # Earth
    x_E, y_E = patch_E.center
    x_E = np.cos((2 * np.pi / 365.2) * i * 10)
    y_E = np.sin((2 * np.pi / 365.2) * i * 10)
    patch_E.center = (x_E, y_E)
    # Jupiter
    x_J, y_J = patch_J.center
    x_J = 5.2 * np.cos((2 * np.pi / 4383) * i * 10 + (np.pi * alpha / 180.0))
    y_J = 5.2 * np.sin((2 * np.pi / 4383) * i * 10 + (np.pi * alpha / 180.0))
    patch_J.center = (x_J, y_J)
    # Hohmann
    # Ellipse Equation r(ß) = a*(1 - e^2)/(1 + e*cos (ß))
    x_H = (
        a
        * (1.0 - e**2)
        / (1.0 + e * np.cos((2 * np.pi / Period) * i * 10))
        * np.cos((2 * np.pi / Period) * i * 10)
    )
    y_H = (
        a
        * (1.0 - e**2)
        / (1.0 + e * np.cos((2 * np.pi / Period) * i * 10))
        * np.sin((2 * np.pi / Period) * i * 10)
    )
    patch_H.center = (x_H, y_H)
    return (
        patch_E,
        patch_J,
        patch_H,
    )


rcParams["animation.convert_path"] = r"C:\Program Files\ImageMagick\convert"
rcParams["animation.ffmpeg_path"] = (
    r"C:\FFmpeg\bin\ffmpeg.exe"  # converting the animation into a gif
)
fig, ax = plt.subplots(
    figsize=(12, 10)
)  # config and plots the whole animation

fig.subplots_adjust(left=0.15, bottom=0.16, right=0.99, top=0.97)
ax.set_facecolor("black")
ax.plot(
    0,
    0,
    color="orange",
    marker="o",
    linestyle="",
    markersize=16,
    markerfacecolor="yellow",
    label="Sun",
)
ax.plot([], [], color="blue", linestyle="", marker="o", label="Earth")
ax.plot([], [], color="green", linestyle="", marker="o", label="Jupiter")
ax.plot([], [], color="red", linestyle="", marker="o", label="spacecraft")
ax.add_patch(Earth)  # puts the on the corner the labes
ax.add_patch(Jupiter)
ax.set_xlabel("X [AU]", fontsize=12)
ax.set_ylabel("Y [AU]", fontsize=12)
ax.legend(loc="best", fontsize=12)
anim = animation.FuncAnimation(
    fig, func=animate, init_func=init, frames=360, interval=10, blit=True
)  # animates the code frames = 200
plt.axis("scaled")  # Scale the plot in real time
# anim.save('Hohmann_Jupiter.gif', writer='imagemagick', fps = 60, bitrate=1800 ) # saves the
# animation as gif

plt.show()

# Convert COE to R & V<font size=2>

2024-03-04, resetta code, https://rosettacode.org/wiki/Orbital_elements#Python  
When neglecting the influence of other objects, two celestial bodies orbit one another along a conic trajectory. In the orbital plane, the polar equation is thus:

r = L/(1 + e cos(angle))

L , e and angle are respectively called semi-latus rectum, eccentricity and true anomaly. The eccentricity and the true anomaly are two of the six so-called orbital elements often used to specify an orbit and the position of a point on this orbit.

The four other parameters are the semi-major axis, the longitude of the ascending node, the inclination and the argument of periapsis. An other parameter, called the gravitational parameter, along with dynamical considerations described further, also allows for the determination of the speed of the orbiting object.

The semi-major axis is half the distance between perihelion and aphelion. It is often noted a, and it's not too hard to see how it's related to the semi-latus-rectum:

a = L/(1 - e2)

The longitude of the ascending node, the inclination and the argument of the periapsis specify the orientation of the orbiting plane with respect to a reference plane defined with three arbitrarily chosen reference distant stars.

The gravitational parameter is the coefficent GM in Newton's gravitational force. It is sometimes noted µ and will be chosen as one here for the sake of simplicity:

µ = GM = 1

As mentioned, dynamical considerations allow for the determination of the speed. They result in the so-called vis-viva equation:

v2 = GM(2/r - 1/a)

This only gives the magnitude of the speed. The direction is easily determined since it's tangent to the conic.

Those parameters allow for the determination of both the position and the speed of the orbiting object in cartesian coordinates, those two vectors constituting the so-called orbital state vectors.

Task
Show how to perform this conversion from orbital elements to orbital state vectors in your programming language.


In [11]:
# Convert
import math


class Vector:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y, self.z + other.z)

    def __mul__(self, other):
        return Vector(self.x * other, self.y * other, self.z * other)

    def __div__(self, other):
        return Vector(self.x / other, self.y / other, self.z / other)

    def __str__(self):
        return "({x}, {y}, {z})".format(x=self.x, y=self.y, z=self.z)

    def abs(self):
        return math.sqrt(self.x * self.x + self.y * self.y + self.z * self.z)


def mulAdd(v1, x1, v2, x2):
    return v1 * x1 + v2 * x2


def rotate(i, j, alpha):
    return [
        mulAdd(i, math.cos(alpha), j, math.sin(alpha)),
        mulAdd(i, -math.sin(alpha), j, math.cos(alpha)),
    ]


def orbitalStateVectors(
    semimajorAxis,
    eccentricity,
    inclination,
    longitudeOfAscendingNode,
    argumentOfPeriapsis,
    trueAnomaly,
):
    i = Vector(1, 0, 0)
    j = Vector(0, 1, 0)
    k = Vector(0, 0, 1)

    p = rotate(i, j, longitudeOfAscendingNode)
    i = p[0]
    j = p[1]
    p = rotate(j, k, inclination)
    j = p[0]
    p = rotate(i, j, argumentOfPeriapsis)
    i = p[0]
    j = p[1]

    l = 2.0 if (eccentricity == 1.0) else 1.0 - eccentricity * eccentricity
    l *= semimajorAxis
    c = math.cos(trueAnomaly)
    s = math.sin(trueAnomaly)
    r = 1 / (1.0 + eccentricity * c)
    rprime = s * r * r / l
    position = mulAdd(i, c, j, s) * r
    speed = mulAdd(i, rprime * c - r * s, j, rprime * s + r * c)
    speed = speed / speed.abs()
    speed = speed * math.sqrt(2.0 / r - 1.0 / semimajorAxis)

    return [position, speed]


ps = orbitalStateVectors(1.0, 0.1, 0.0, 355.0 / (113.0 * 6.0), 0.0, 0.0)
print("Position :", ps[0])
print("Speed    :", ps[1])

TypeError: unsupported operand type(s) for /: 'Vector' and 'float'